In [3]:
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
def perform_df_clean(raw_df):

    df = raw_df.copy()
    
    # Rename columns to snake_case
    df.columns = (
        df.columns.str.strip()
                 .str.lower()
                 .str.replace(r'[^\w\s]', '', regex=True)
                 .str.replace(r'\s+', '_', regex=True)
    )

    df = df.drop(columns=['parking'])
    
    # Drop columns where more than 50% of values are NaN
    df = df.dropna(axis=1, thresh=int(0.5 * len(df)))
    
    # Extract postcodes
    postcode_pattern = r'([GI]{2}R 0[A]{2})|(([A-Z][0-9]{1,2})|([A-Z][A-HJ-Y][0-9]{1,2})|([A-Z][0-9][A-Z])|([A-Z][A-HJ-Y][0-9][A-Z]?))'
    df.loc[:, 'postcode'] = df['address'].apply(lambda x: re.search(postcode_pattern, x).group() if re.search(postcode_pattern, x) else np.nan)
    
    # Cast ints
    df.loc[:, 'bedrooms'] = df['bedrooms'].astype('Int64', errors='ignore')
    df.loc[:, 'bathrooms'] = df['bathrooms'].astype('Int64', errors='ignore')
    
    # Parse amounts
    df.loc[:, 'price_pcm'] = df['price_pcm'].replace({'£': '', ',': '', ' pcm': ''}, regex=True).astype(float)
    if 'price_per_week' in df.columns:
        df.loc[:, 'price_per_week'] = df['price_per_week'].replace({'£': '', ',': '', ' pw': ''}, regex=True).astype(float)

    if 'let_available_date' in df.columns:
        # Convert dates
        now = datetime.today().strftime('%d/%m/%Y')
        df.loc[:, 'let_available_date'] = df['let_available_date'].replace('Now', now)
        df.loc[:, 'let_available_date'] = pd.to_datetime(df['let_available_date'], format='%d/%m/%Y', errors='coerce')
    
    # Categorize
    
    category_cols = ['property_type', 'postcode']
    
    if 'council_tax' in df.columns:
        df.loc[:, 'council_tax'] = df['council_tax'].replace({'Band: ': ''}, regex=True)
        category_cols.append('council_tax')
    if 'furnish_type' in df.columns:
        category_cols.append('furnish_type')
    if 'tenure' in df.columns:
        category_cols.append('tenure')
    if 'let_type' in df.columns:
        category_cols.append('let_type')
        
    df.loc[:, category_cols] = df[category_cols].astype('category')

    return df

In [5]:
def clean_dfs(parent_folder):

    for location in os.listdir(parent_folder):
        location_path = os.path.join(parent_folder, location)
        
        if os.path.isdir(location_path):
            csv_file_path = os.path.join(location_path, 'data.csv')
            
            if os.path.exists(csv_file_path):
                raw_df = pd.read_csv(csv_file_path, na_values=['Ask agent', 'POA'])
                clean_df = perform_df_clean(raw_df)
                output_csv = os.path.join(location_path, 'data_clean.csv')
                clean_df.to_csv(output_csv, index=False)
                os.chmod(output_csv, 0o444) # Make file read-only
                print(f'Cleaned file saved: {output_csv}')

In [6]:
clean_dfs('rent')
clean_dfs('sale')

Cleaned file saved: rent/Nottingham/data_clean.csv
Cleaned file saved: rent/Newcastle-Under-Lyme/data_clean.csv
Cleaned file saved: rent/Lancaster/data_clean.csv
Cleaned file saved: rent/York/data_clean.csv
Cleaned file saved: rent/Loughborough/data_clean.csv
Cleaned file saved: sale/Nottingham/data_clean.csv
Cleaned file saved: sale/Newcastle-Under-Lyme/data_clean.csv
Cleaned file saved: sale/Loughborough/data_clean.csv
Cleaned file saved: sale/Lancaster/data_clean.csv
Cleaned file saved: sale/York/data_clean.csv
